In [3]:
import cv2
import sys

(major_ver, minor_ver, subminor_ver) = (cv2.__version__).split('.')

if __name__ == '__main__' :

    # Set up tracker.
    # Instead of MIL, you can also use

    tracker_types = ['BOOSTING', 'MIL','KCF', 'TLD', 'MEDIANFLOW', 'GOTURN', 'MOSSE', 'CSRT']
    tracker_type = tracker_types[2]

    if int(minor_ver) < 3:
        tracker = cv2.Tracker_create(tracker_type)
    else:
        if tracker_type == 'BOOSTING':
            tracker = cv2.TrackerBoosting_create()
        if tracker_type == 'MIL':
            tracker = cv2.TrackerMIL_create()
        if tracker_type == 'KCF':
            tracker = cv2.TrackerKCF_create()
        if tracker_type == 'TLD':
            tracker = cv2.TrackerTLD_create()
        if tracker_type == 'MEDIANFLOW':
            tracker = cv2.TrackerMedianFlow_create()
        if tracker_type == 'GOTURN':
            tracker = cv2.TrackerGOTURN_create()
        if tracker_type == 'MOSSE':
            tracker = cv2.TrackerMOSSE_create()
        if tracker_type == "CSRT":
            tracker = cv2.TrackerCSRT_create()

    # Read video
    video = cv2.VideoCapture("/home/ubuntu/Desktop/parag/Face_mask_detection/vehicle-speed-check/cars.mp4")

    # Exit if video not opened.
    if not video.isOpened():
        print ("Could not open video")
        sys.exit()

    # Read first frame.
    ok, frame = video.read()
    if not ok:
        print ('Cannot read video file')
        sys.exit()
    
    # Define an initial bounding box
    bbox = (287, 23, 86, 320)

    # Uncomment the line below to select a different bounding box
    bbox = cv2.selectROI(frame, False)

    # Initialize tracker with first frame and bounding box
    ok = tracker.init(frame, bbox)

    while True:
        # Read a new frame
        ok, frame = video.read()
        if not ok:
            break
        
        # Start timer
        timer = cv2.getTickCount()

        # Update tracker
        ok, bbox = tracker.update(frame)

        # Calculate Frames per second (FPS)
        fps = cv2.getTickFrequency() / (cv2.getTickCount() - timer);

        # Draw bounding box
        if ok:
            # Tracking success
            p1 = (int(bbox[0]), int(bbox[1]))
            p2 = (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3]))
            cv2.rectangle(frame, p1, p2, (255,0,0), 2, 1)
        else :
            # Tracking failure
            cv2.putText(frame, "Tracking failure detected", (100,80), cv2.FONT_HERSHEY_SIMPLEX, 0.75,(0,0,255),2)

        # Display tracker type on frame
        cv2.putText(frame, tracker_type + " Tracker", (100,20), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (50,170,50),2);
    
        # Display FPS on frame
        cv2.putText(frame, "FPS : " + str(int(fps)), (100,50), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (50,170,50), 2);

        # Display result
        cv2.imshow("Tracking", frame)

        # Exit if ESC pressed
        k = cv2.waitKey(1) & 0xff
        if k == 27 : break


## Dusra code

In [2]:
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
# cap = cv2.VideoCapture('/home/ubuntu/Desktop/parag/Face_mask_detection/vehicle-speed-check/cars.mp4')
cap = cv2.VideoCapture('/home/ubuntu/Desktop/MTW-CCTVVideos/lift.mp4')

font=cv2.FONT_HERSHEY_SIMPLEX
ret, frame = cap.read()
frame=frame[:,200:]
fgbg = cv2.bgsegm.createBackgroundSubtractorMOG()
polygon1 = Polygon([[42,316],[173,297],[210,348],[63,361]])
kernel1 = np.array([[1,1],[1,1]])
fcount=0
a=[]
r=0
speed=0
count=0
c=0
#cap.get(3),cap.get(4))
OverSpeed=[]
fourc=cv2.VideoWriter_fourcc(*'XVID')
out1=cv2.VideoWriter('home/ubuntu/Desktop/parag/speed.mp4',fourc,30.0,(640,480))
out2=cv2.VideoWriter('home/ubuntu/Desktop/parag/shadow.mp4',fourc,30.0,(640,480))
out3=cv2.VideoWriter('home/ubuntu/Desktop/parag/wshadow.mp4',fourc,30.0,(640,480))
def shadow(frame):
    gimg=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    _,th=cv2.threshold(gimg,140,255,cv2.THRESH_BINARY)
    fg=cv2.bitwise_and(th,th,mask=fgmask)
    return fg
while(ret):
    fcount=fcount+1
    ret, frame = cap.read()
    frame=frame[:,200:]
    img=frame.copy()
    pts1=np.array([[42,316],[173,297],[210,348],[63,361]],np.int32)
    pts1=pts1.reshape((-1,1,2))
    cv2.polylines(frame,[pts1],1,(0,255,255))
    fgmask = fgbg.apply(frame)
    shad=shadow(frame)
    cv2.imshow('window1',fgmask)
    cv2.imshow('window2',shad)
    out2.write(fgmask)
    out3.write(shad)
    fgmask = cv2.morphologyEx(fgmask, cv2.MORPH_CLOSE, kernel1)
    fgmask=cv2.erode(fgmask,kernel1,iterations=2)
    fgmask=cv2.dilate(fgmask, kernel1, iterations=10)
    gimg=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    _,th=cv2.threshold(gimg,140,255,cv2.THRESH_BINARY)
    fg=cv2.bitwise_and(th,th,mask=fgmask)
    img[:,:,2]=cv2.add(img[:,:,2],fg)
    #cv2.imshow('red',img)
    #cv2.imshow('window2',fg)
    ## speed and contour ##
    (cnts,_)=cv2.findContours(fg.copy(),cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for contour in cnts:
        (x, y, w, h)=cv2.boundingRect(contour)
        if w>=30 and h>=30:
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255,0,0), 1)
            cv2.circle(frame,(int(x+w/2),int(y+(h)/2)),5,(0,0,255),-1)
            if(polygon1.contains(Point(int(x+w/2),int(y+(h)/2)))):
                #print('yes')
                a.append(fcount)
            else:
                pass
                #print('NO')
    dist=4.6
    fps=30
    if len(a)>1:
        if fcount-max(a)>1:
            #print(a)
            speed=((dist*fps)/(max(a)-min(a)))*(18/5)
            count=count+1
            print(a,speed)
            a=[]
    s="SPEED="+str(round(speed,2))
    cv2.putText(frame,s,(0,100),font,1,(0,255,0),2,cv2.LINE_AA)
    out1.write(frame)
    cv2.imshow('window3',frame)
    k = cv2.waitKey(1)
    if k == ord('q'):
        break
cap.release()
out1.release()
out2.release()
out3.release()
cv2.destroyAllWindows()q

[982, 983, 984, 985, 986, 987, 988, 989, 990, 991, 992, 993, 994, 995, 996, 997, 998, 999, 1000, 1001, 1002] 24.840000000000003
[1013, 1014, 1015, 1016, 1017, 1018, 1019, 1020, 1021, 1022, 1023, 1024, 1025, 1026, 1027, 1028, 1029, 1030, 1031, 1032, 1033, 1034, 1035, 1036, 1037] 20.7
[1047, 1048] 496.8
[1167, 1170, 1171, 1172, 1173, 1174, 1175, 1176, 1177, 1178, 1179, 1180] 38.215384615384615
[1183, 1184, 1185, 1186, 1187, 1188, 1190] 70.97142857142858
[1196, 1197, 1198, 1199] 165.6
[1212, 1213, 1214, 1215, 1216, 1217, 1218, 1219, 1220, 1221, 1222, 1223] 45.163636363636364
[1234, 1235, 1236, 1237, 1238, 1239, 1240, 1241] 70.97142857142858
[1253, 1254, 1255] 248.4
[1259, 1260] 496.8
[1270, 1271, 1272, 1273] 165.6
[1609, 1610, 1611, 1612] 165.6
[1624, 1625, 1626, 1627, 1628, 1629, 1630, 1631, 1632, 1633, 1634, 1635, 1636, 1637, 1638, 1639, 1640, 1641, 1642, 1643, 1644, 1645, 1646, 1647, 1648, 1649] 19.872


In [4]:
!pip install shapely

     |████████████████████████████████| 1.0 MB 11.4 MB/s eta 0:00:01
